In [0]:
-- ADS_SMA_CPP_MAIN_PREPARE - Part1

--insert /*+ APPEND PARALLEL(8) */  into STG_SMA_CASE_PHASE_PROPERTIES_parse  
  select 
    upper(attr_fp)||'.'||case_cidla cpprop_source_id,
  case when INSTR(attr_fp, '.', -1)>0 then SUBSTR (attr_fp,
                       0,
                       (INSTR (attr_fp, '.', -1)) - 1) ||'.'||case_cidla
                       else null end  cpprop_parent_source_id,
    CASE_CIDLA as case_source_id,
    upper(attr_fp)||'|'||casetype as cpptp_source_id ,
  upper(ATTR_FP) as attr_fp ,
  CASETYPE   ,
  ATTR_VALUE  ,
  ATTR_BVALUE  ,
  ATTR_TYPE  ,
  STRUCTURETYPE ,
  MAPPING, 
  timestamp
   from (
  select 
    att1.eventid,
  att1.case_cidla,
  att1.casetype,
  att1.timestamp,
  att1.attr_id,
  upper(att1.attr_fp) Attr_FP,
  upper(att1.attr_name) Attr_name,
  att1.attr_value,
  att1.attr_bvalue,
  att1.attr_type,
  att1.parentid,
  --row_number() over (partition by  upper(att1.attr_fp),att1.case_cidla order by att1.timestamp desc) as rn,
  att1.modifiedby, 
  att1.structuretype,  
  att1.mapping
  from (select 
       att0.eventid,
       att0.type,
       att0.case_cidla,
       att0.casetype,
       att0.timestamp,
      attr.*--,
     -- row_number() over (partition by att0.case_cidla, attr.attr_fp order by timestamp desc)
      from (select 
       j.eventid,
       j.cidla as case_cidla,
       case when j.casetype like '% %' then 'ERR' else j.casetype end as casetype,
       j.type,
       j.timestamp,
       jsondata
      from ADS_ETL_OWNER.DLK_ADS_SMA_MONITOR_EVENTS_FULL j
      where
      --CASE filter
        j.type in ('CASE_CREATED',
    'CASE_COMPLETED',
    'CASE_WAITING',
    'CASE_RUNNING',
    'CASE_UPDATED',
    'CASE_PHASE_CHANGED',
    'CASE_RECONCILE'
--,     --JVE:  zmena typu case
          )
       and unix_ts_to_date(json_value(jsondata, '$.theCase.createdAt')/1000) >= ADD_MONTHS(trunc(SYSDATE), -25) 
          )att0 
      ,
      json_table(jsondata, '$.theCase.attributes[*]' 
      columns (attr_id path '$.id',
      attr_fp path '$.fullPath',
      attr_name path '$.name',
      attr_value path '$.value',
      attr_bvalue clob path '$.blobValue',
      attr_type path '$.attributeType',
      parentid path '$.parentId',
      modifiedby path '$.modifiedBy',
      structuretype path '$.structureType',
      mapping path '$.mapping')
      ) attr) att1)-- where --rn=1
;
SELECT get_json_object(jsondata, '$.theCase.attributes') 
FROM gap_catalog.ads_src_20250901.DLK_ADS_SMA_MONITOR_EVENTS_FULL 
LIMIT 5;
----------------------------------------------------------------------------
WITH base_cases AS (
    SELECT 
        j.eventid,
        j.cidla as case_cidla,
        CASE 
            WHEN j.casetype LIKE '% %' THEN 'ERR' 
            ELSE j.casetype 
        END as casetype,
        j.type,
        j.timestamp,
        j.jsondata
    FROM gap_catalog.ads_src_20250901.DLK_ADS_SMA_MONITOR_EVENTS_FULL j
    WHERE 
        j.type IN (
            'CASE_CREATED', 'CASE_COMPLETED', 'CASE_WAITING',
            'CASE_RUNNING', 'CASE_UPDATED', 'CASE_PHASE_CHANGED', 'CASE_RECONCILE'
        )
        AND FROM_UNIXTIME(
            CAST(get_json_object(jsondata, '$.theCase.createdAt') AS BIGINT) / 1000
        ) >= ADD_MONTHS(CURRENT_DATE(), -25)
)
SELECT 
    UPPER(attr.fullPath) || '.' || bc.case_cidla as cpprop_source_id,
    CASE 
        WHEN INSTR(attr.fullPath, '.') > 0 
        THEN SUBSTRING(
            attr.fullPath, 
            1, 
            LENGTH(attr.fullPath) - LOCATE('.', REVERSE(attr.fullPath))
        ) || '.' || bc.case_cidla
        ELSE NULL 
    END as cpprop_parent_source_id,
    bc.case_cidla as case_source_id,
    UPPER(attr.fullPath) || '|' || bc.casetype as cpptp_source_id,
    UPPER(attr.fullPath) as attr_fp,
    bc.casetype,
    attr.value as attr_value,
    attr.blobValue as attr_bvalue,
    attr.attributeType as attr_type,
    attr.structureType as structuretype,
    attr.mapping,
    bc.timestamp
FROM base_cases bc
LATERAL VIEW EXPLODE(
    FROM_JSON(
        get_json_object(jsondata, '$.theCase.attributes'),
        'array<struct<
            id:string,
            fullPath:string,
            name:string,
            value:string,
            blobValue:string,
            attributeType:string,
            parentId:string,
            modifiedBy:string,
            structureType:string,
            mapping:string
        >>'
    )
) attr_table AS attr;